In [ ]:
%gui qt

import logging

from pyqtgraph.flowchart import Flowchart
from pyqtgraph import Qt
from pyqtgraph.Qt import QtGui, QtCore
from pyqtgraph.flowchart import library as fclib
from pyqtgraph.dockarea import Dock, DockArea

from plottr.data.datadict import togrid, DataDict, GridDataDict
from plottr import log as plottrlog
from plottr.node.data_selector import DataSelector
from plottr.node.dim_reducer import XYAxesSelector
from plottr.plot.mpl import PlotNode, AutoPlot

import test_data

In [ ]:
nodelib = fclib.NodeLibrary()
nodelib.addNodeType(DataSelector, [('Basic')])
nodelib.addNodeType(XYAxesSelector, [('Basic')])
nodelib.addNodeType(PlotNode, [('Plot')])

### Setting up the flowchart

# library
fc = Flowchart(terminals={
    'dataIn': {'io': 'in'},
    'dataOut': {'io': 'out'}
})
fc.library = nodelib

# flowchart for this example
datasel = fc.createNode('DataSelector')
xysel = fc.createNode('XYAxesSelector')
plot = fc.createNode('Plot')

fc.connectTerminals(fc['dataIn'], datasel['dataIn'])
fc.connectTerminals(datasel['dataOut'], xysel['dataIn'])
fc.connectTerminals(xysel['dataOut'], fc['dataOut'])
fc.connectTerminals(xysel['dataOut'], plot['dataIn'])

plotWidget = AutoPlot()
plot.setPlotWidget(plotWidget)


### Setting up the GUI
win = QtGui.QMainWindow()
area = DockArea()
win.setCentralWidget(area)

dataselDock = Dock('Data Selector')
dataselDock.addWidget(datasel.ui)
area.addDock(dataselDock)

# xy selector
xyselDock = Dock('XY Axes Selector')
xyselDock.addWidget(xysel.ui)
area.addDock(xyselDock, 'bottom', dataselDock)

# logger
logDock = Dock('Log')
logDock.addWidget(plottrlog.setupLogging(makeDialog=False))
area.addDock(logDock, 'bottom', xyselDock)

# plot
plotDock = Dock('Plot')
plotDock.addWidget(plotWidget)
area.addDock(plotDock, 'right')

win.show()

### Logging
logger = logging.getLogger('plottr.interactive')
plottrlog.LEVEL = logging.DEBUG

In [ ]:
data = test_data.three_incompatible_3d_sets(21, 21, 21)
fc.setInput(dataIn=data)

In [ ]:
# datasel.selectedData = ['data', 'more_data']

In [ ]:
# xysel.xyAxes = ('x', 'y')

In [ ]:
fc.output()